In [6]:
search_for = 112
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.008468866348266602
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 350
all cases: 562404387
type: [1, 1, 1, 1, 112] 112
cases of this type: 157351936
10000 53.40994003232332
20000 48.68729874261557
30000 59.32765013740576
40000 91.94198009337248
50000 288.74728075499127
60000 236.90524776447242
70000 218.5053886700002
80000 213.62628895568497
90000 192.3289048369553
100000 256.8805285527588
110000 200.32377009890305
120000 207.7541985808047
130000 204.59945354418485
140000 240.35166128805164
150000 254.87562529820042
160000 281.0676837570692
170000 316.74543328268567
180000 395.9401339171135
190000 458.4047711096639
200000 483.39000098519483
210000 387.1711963012148
220000 484.44944251397067
230000 498.168284087305
240000 449.815009815999
250000 379.03979947223144
260000 465.09388182340814
270000 480.2149157734529
280000 453.73242475599756
290000 400.22243881914676
300000 416.28538278778626
310000 391.238409

3070000 374.1962142821926
3080000 416.6356655398611
3090000 412.2780496279076
3100000 445.0280145728085
3110000 414.4760759826836
3120000 428.56503355021664
3130000 380.7424765082015
3140000 425.4722990370348
3150000 424.2188345360754
3160000 462.7842852713051
3170000 464.2299681946205
3180000 412.1472705545406
3190000 416.16153476307073
3200000 442.4346830085402
3210000 429.3461691160426
3220000 391.33290351889394
3230000 410.5655774711102
3240000 399.119959435329
3250000 410.0243072269132
3260000 407.4643280403659
3270000 388.8197011010472
3280000 404.69581813948815
3290000 365.59744422389304
3300000 397.0393177908039
3310000 389.82544843093956
3320000 388.1673038081423
3330000 406.50430942278604
3340000 373.48026332987973
3350000 402.69237252115107
3360000 411.59678713675333
3370000 399.06562061470845
3380000 402.8064626099338
3390000 401.138714033317
3400000 385.1404046328419
3410000 411.1128973957056
3420000 385.7015764719215
3430000 402.8872238184752
3440000 401.3666453385875
345

6190000 419.32009785200546
6200000 380.67727919948743
6210000 410.6444557947156
6220000 405.1419416451471
6230000 394.67383076592665
6240000 475.23061334492706
6250000 418.68133624016
6260000 362.4372233493318
6270000 376.53711863011387
6280000 431.0518610103009
6290000 480.48980738437547
6300000 372.87552325995756
6310000 49.57753355151244
6320000 49.60999346983044
6330000 49.64410205398309
6340000 49.85284890470123
6350000 49.684780625422526
6360000 53.70724599678778
6370000 545.9186302155069
6380000 431.1611593928515
6390000 474.3568307414981
6400000 335.034408375204
6410000 376.63815707978165
6420000 408.77340846381185
6430000 364.5218635685432
6440000 478.79509125635417
6450000 354.03346817691994
6460000 434.4666439896227
6470000 383.69479356314093
6480000 366.3182258350004
6490000 400.71865761671864
6500000 425.13964482050517
6510000 410.4075220914316
6520000 362.5318104736985
6530000 373.1803917720214
6540000 374.5239329098427
6550000 357.64813571378755
6560000 403.7754590972058

9300000 385.9190760662209
9310000 382.4658207118353
9320000 388.2183259689532
9330000 375.9585883803339
9340000 387.49441124958815
9350000 354.0706593854194
9360000 388.6252901083267
9370000 350.0810092470954
9380000 362.52081074552325
9390000 378.79678578865105
9400000 356.1212196339913
9410000 391.6862984740385
9420000 361.54412061927565
9430000 356.9022678753158
9440000 416.7345439994578
9450000 386.10701937626766
9460000 398.0686075020659
9470000 363.8940421552342
9480000 390.8368763650698
9490000 380.500003249022
9500000 373.4198498319519
9510000 420.272325212512
9520000 416.66389433101045
9530000 425.18709281976476
9540000 393.4827502032726
9550000 423.37912949059165
9560000 376.6637318146093
9570000 370.2352026882262
9580000 400.6330637304913
9590000 358.92714694504787
9600000 383.58537132303826
9610000 335.77359301407745
9620000 352.62267691327764
9630000 385.41428019730307
9640000 368.44561494994394
9650000 398.0496170243483
9660000 354.42880471939657
9670000 404.0697131512013

12330000 360.28189895117345
12340000 389.5409494003549
12350000 394.1844126635273
12360000 385.0380730902236
12370000 400.2021850075877
12380000 363.6894023068036
12390000 386.58096895932397
12400000 361.05814577182986
12410000 369.6219061292895
12420000 381.9715049255048
12430000 369.0297926357394
12440000 405.78638080320246
12450000 369.8960014304152
12460000 374.76656778619275
12470000 393.75910005902557
12480000 380.70899024173474
12490000 383.65341461377386
12500000 378.2786743342048
12510000 416.08808587689606
12520000 392.16010065049034
12530000 122.04959173247677
12540000 118.53647607727295
12550000 122.28164883664925
12560000 133.68393260503103
12570000 138.0673457013049
12580000 120.82004578220968
12590000 405.69308443805494
12600000 484.82828782647766
12610000 369.23624326709097
12620000 368.76339654909924
12630000 387.3204028349326
12640000 389.2259756419677
12650000 453.7478914382778
12660000 376.1315172503445
12670000 394.8805877688828
12680000 424.1919669776066
12690000 

...

552850000 1.142384172087974
552860000 1.295729949231545
552870000 1.1987272018347384
552880000 1.4895137552219364
552890000 0.8058355132943326
552900000 0.9389787069280743
552910000 1.480513110191491
552920000 1.3952571404674519
552930000 1.5536833987317284
552940000 1.6496516337096294
552950000 1.4663282870311274
552960000 1.5859394317162434
552970000 1.4976163162089389
552980000 0.7020138258672488
552990000 1.1480368337611555
553000000 1.183141581836813
553010000 1.1452806305223242
553020000 1.3061573725566467
553030000 1.5108928540990616
553040000 1.4779513068752157
553050000 2.0083751504982508
553060000 1.099958745257755
553070000 0.7640059036195609
553080000 0.8802281405246258
553090000 0.9216982761918902
553100000 1.5686323591344689
553110000 1.6226554270307423
553120000 2.0036740812522176
553130000 1.6782642606093618
553140000 1.9151082607461212
553150000 0.9572708489385116
553160000 0.7231626525635058
553170000 1.146417142776072
553180000 1.4286978047706946
553190000 1.20622299

555570000 1.2135151472369232
555580000 1.4147258922084305
555590000 1.3548285812938676
555600000 1.2762596350784898
555610000 0.9109207138524916
555620000 1.057588353043
555630000 1.0604705478664835
555640000 1.0631218645937641
555650000 1.1051987082756427
555660000 1.0138748121730286
555670000 0.7254898331559102
555680000 1.0803424334729645
555690000 0.9330329776269793
555700000 1.0171653123506044
555710000 1.2647758663314248
555720000 1.1984422175193032
555730000 1.1408282262041967
best so far: 0
type: [2, 1, 4, 2, 7] 112
cases of this type: 307328
555740000 1.1119380790154338
555750000 0.8490290989136496
555760000 0.7722840826411181
555770000 0.6268028540168934
555780000 0.8881828447355629
555790000 0.9285894897628757
555800000 0.8230373554702135
555810000 0.9093068002341588
555820000 0.8980302219686509
555830000 0.6146927639786005
555840000 0.5169398750318487
555850000 0.46937389350210296
555860000 0.5057142852360673
555870000 0.49724554881983996
555880000 0.5055200797575778
555890

558020000 0.8636344005410538
558030000 0.5709248541650176
558040000 1.147376248667419
558050000 0.6700931856684619
558060000 1.0435358685335319
best so far: 0
type: [2, 2, 1, 2, 14] 112
cases of this type: 614656
558070000 0.8195719869386554
558080000 0.77880902731863
558090000 0.5181303692930936
558100000 0.23821226814721025
558110000 0.9413354969907205
558120000 0.7317037132866382
558130000 0.7645865058103667
558140000 0.29076450912275575
558150000 0.5280634334755142
558160000 0.7728285146664116
558170000 0.6896877149101696
558180000 0.3977831377533078
558190000 0.47180216027890975
558200000 0.6919264678226908
558210000 0.7892455067555904
558220000 0.5770907944296598
558230000 0.3506191122541428
558240000 0.6090333094118039
558250000 0.7780536790647838
558260000 0.666681471704834
558270000 0.34914057562478384
558280000 0.4408513679771291
558290000 0.756738000210219
558300000 0.8839150178752542
558310000 0.5317721685711807
558320000 0.3577539419928723
558330000 0.699244346315066
55834

best so far: 0
type: [2, 4, 7, 2, 1] 112
cases of this type: 1568
best so far: 0
type: [2, 4, 14, 1, 1] 112
cases of this type: 784
560280000 0.12688416249614953
best so far: 0
type: [2, 7, 1, 1, 8] 112
cases of this type: 28672
560290000 0.21616723817148473
560300000 0.11098511468758186
best so far: 0
type: [2, 7, 1, 2, 4] 112
cases of this type: 14336
560310000 0.16972064392668007
560320000 0.1667583795618415
best so far: 0
type: [2, 7, 1, 4, 2] 112
cases of this type: 7168
560330000 0.1096807581171261
best so far: 0
type: [2, 7, 1, 8, 1] 112
cases of this type: 3584
best so far: 0
type: [2, 7, 2, 1, 4] 112
cases of this type: 7168
560340000 0.1501317748145064
best so far: 0
type: [2, 7, 2, 2, 2] 112
cases of this type: 3584
best so far: 0
type: [2, 7, 2, 4, 1] 112
cases of this type: 1792
best so far: 0
type: [2, 7, 4, 1, 2] 112
cases of this type: 1792
best so far: 0
type: [2, 7, 4, 2, 1] 112
cases of this type: 896
best so far: 0
type: [2, 7, 8, 1, 1] 112
cases of this type: 448
b

562030000 0.04345990640464094
562040000 0.028898374594072504
562050000 0.027716138876179856
562060000 0.031700657775958374
best so far: 0
type: [4, 2, 1, 7, 2] 112
cases of this type: 10976
562070000 0.02262000947897964
best so far: 0
type: [4, 2, 1, 14, 1] 112
cases of this type: 5488
best so far: 0
type: [4, 2, 2, 1, 7] 112
cases of this type: 19208
562080000 0.025115698838233946
562090000 0.0219940038171543
best so far: 0
type: [4, 2, 2, 7, 1] 112
cases of this type: 2744
best so far: 0
type: [4, 2, 7, 1, 2] 112
cases of this type: 1568
562100000 0.018288931201146708
best so far: 0
type: [4, 2, 7, 2, 1] 112
cases of this type: 784
best so far: 0
type: [4, 2, 14, 1, 1] 112
cases of this type: 392
best so far: 0
type: [4, 4, 1, 1, 7] 112
cases of this type: 9604
562110000 0.022720896323621273
best so far: 0
type: [4, 4, 1, 7, 1] 112
cases of this type: 1372
best so far: 0
type: [4, 4, 7, 1, 1] 112
cases of this type: 196
best so far: 0
type: [4, 7, 1, 1, 4] 112
cases of this type: 179